# CSE 6040, Fall 2015 [24]: "Online" regression

This notebook continues the linear regression problem from last time, but asks about a method that can estimate the regression coefficients when you only get to see samples "one-at-a-time." We refer to such a fitting procedure as being "online," rather than "offline" ("batched").

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Scalability with the problem size

In [ ]:
def generate_model (d):
    """Returns a set of d+1 linear model coefficients."""
    return np.random.rand (d+1, 1)

def generate_data (m, x, sigma=2.**0.5):
    """
    Generates 'm' noisy observations for a linear model whose
    predictor (non-intercept) coefficients are given in 'x'.
    """
    assert (type (x) is np.ndarray) and (x.ndim == 2) and (x.shape[1] == 1)
    n = len (x)
    A = np.random.rand (m, n)
    A[:, 0] = 1.0
    b = A.dot (x) + sigma*np.random.randn (m, 1)
    return (A, b)

def estimate_coeffs (A, b):
    """
    Solves Ax=b by a linear least squares method.
    """
    result = np.linalg.lstsq (A, b)
    x = result[0]
    return x

In [ ]:
# Demo the above routines for a 2-D dataset.

m = 50
x_true = generate_model (1)
(A, b) = generate_data (m, x_true)

print A.shape
print x_true.shape
print b.shape

print "Condition number of the data matrix: ", np.linalg.cond (A)
print "True solution:", x_true.T

x = estimate_coeffs (A, b)

print "Computed solution:", x.T

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.plot (A[:, 1], b, 'b+') # Noisy observations
ax1.plot (A[:, 1], A.dot (x), 'r*') # Fit
ax1.plot (A[:, 1], A.dot (x_true), 'go') # True solution

In [ ]:
# Benchmark, as 'm' varies:

n = 32 # dimension
M = [100, 1000, 10000, 100000, 1000000]
times = [0.] * len (M)
for (i, m) in enumerate (M):
    x_true = generate_model (n)
    (A, b) = generate_data (m, x_true)
    t = %timeit -o estimate_coeffs (A, b)
    times[i] = t.best

In [ ]:
t_linear = [times[0]/M[0]*m for m in M]

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.loglog (M, times, 'bo')
ax1.loglog (M, t_linear, 'r--')

In [ ]:
N = [2, 4, 8, 16, 32, 64, 128, 256]
m = 100000
times = [0.] * len (N)
for (i, n) in enumerate (N):
    x_true = generate_model (n)
    (A, b) = generate_data (m, x_true)
    t = %timeit -o estimate_coeffs (A, b)
    times[i] = t.best

In [ ]:
t_linear = [times[0]/N[0]*n for n in N]
t_quadratic = [times[0]/N[0]/N[0]*n*n for n in N]

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.loglog (N, times, 'bo')
ax1.loglog (N, t_linear, 'r--')
ax1.loglog (N, t_quadratic, 'g--')

In [ ]:
m = 100000
d = 1
x_true = generate_model (d)
(A, b) = generate_data (m, x_true)

print "Condition number of the data matrix: ", np.linalg.cond (A)

x = estimate_coeffs (A, b)
r = x - x_true

e_abs = np.linalg.norm (np.abs (r), ord=2)
e_rel = e_abs / np.linalg.norm (np.linalg.norm (x_true))

print "Absolute error:", e_abs
print "Relative error:", e_rel

%timeit estimate_coeffs (A, b)

In [ ]:
lambda_max = max (np.linalg.eigvals (A.T.dot (A)))
print lambda_max

In [ ]:
MU = 2.0 / lambda_max
X = np.zeros ((d+1, m+1))
X[:, 0] = A[0, :].T

# @YOUSE: FILL IN ONLINE ALGORITHM HERE

In [ ]:
def rel_diff (x, y):
    return np.linalg.norm (x - y) / np.linalg.norm (y)

rel_errs = [rel_diff (X[:, k], x_true) for k in range (m+1)]
print x_true.T
print x.T
print X[:, m]

In [ ]:
plt.plot (range (m+1), rel_errs)

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.plot (A[:, 1], b, 'b+')
ax1.plot (A[:, 1], A.dot (x_true), 'r*')
ax1.plot (A[:, 1], A.dot (X[:, m]), 'go')